In [ ]:
import pygame
import pymunk
from pymunk.vec2d import Vec2d
import pymunk.pygame_util
import math
import neat
import os
import pickle
import random

# --- Constants ---
WIDTH, HEIGHT = 800, 600
ROBOT_RADIUS = 15
MAX_SENSOR_RANGE = 100
NUM_SENSORS = 7 # Increased number of sensors for more detailed input
SENSOR_ANGLE_SPREAD = math.pi * 3 / 4 # Wider spread for sensors
DRAW = False  # Toggle this to True to visualize training (set to True when visualizing best genome)

# --- Pygame Initialization ---
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
clock = pygame.time.Clock()
font = pygame.font.SysFont("Arial", 18)

# --- Colors ---
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)

# --- PyMunk Space and Options ---
space = None  # Will be initialized per genome evaluation
draw_options = pymunk.pygame_util.DrawOptions(screen)

# --- Collision Types ---
ROBOT_COLLISION_TYPE = 1
WALL_COLLISION_TYPE = 2 # This will now also include randomized obstacles

# --- Obstacle Generation Parameters ---
MIN_INNER_OBSTACLES = 5 # Increased for more varied and challenging environments
MAX_INNER_OBSTACLES = 10 # Increased
OBSTACLE_MIN_SIZE = 50
OBSTACLE_MAX_SIZE = 300 # Increased for potentially longer walls
OBSTACLE_THICKNESS = 10 # For rectangle obstacles

# --- Fitness Calculation Parameters ---
MIN_DESIRED_SPEED = 15.0 # Tune this value! (e.g., 5.0 to 20.0)
EXPLORATION_GRID_SIZE = 50 # Size of grid cells for exploration reward
EXPLORATION_REWARD_PER_CELL = 10.0 # Reward for entering a new grid cell

# --- Functions ---
def show_intro():
    """Displays an introductory screen with information about the NEAT simulation."""
    intro_text = [
        "NEAT Simulation Overview:",
        "Start Simple - Small population with simple brains (networks)",
        "Test and Score - Each brain drives a robot and earns points for distance without crashing",
        "Pick the Fittest - Best brains pass genes (structure + weights) to next generation",
        "Mix and Mutate - Combine parents, mutate weights, add nodes/connections",
        "Protect New Ideas - NEAT uses species to preserve innovation",
        "Repeat - Over generations, brains evolve to skillfully drive the robot",
        "",
        "Obstacles will be randomized and varied in number each generation!",
        "The fitness function now penalizes excessive turning and rewards exploration.",
        "Press any key to begin..."
    ]
    screen.fill(WHITE)
    for i, line in enumerate(intro_text):
        rendered = font.render(line, True, BLACK)
        screen.blit(rendered, (20, 30 + i * 30))
    pygame.display.flip()
    waiting = True
    while waiting:
        for event in pygame.event.get():
            if event.type == pygame.KEYDOWN or event.type == pygame.QUIT:
                waiting = False

def create_pymunk_obstacles(current_space, num_inner_obstacles):
    """
    Creates and adds static obstacles (borders and inner randomized rectangles)
    to the given PyMunk space.
    Returns a list of pymunk shapes and a list of their pygame rect equivalents.
    """
    pymunk_shapes = []

    # Fixed border walls
    border_rects = [
        pygame.Rect(0, 0, WIDTH, OBSTACLE_THICKNESS),           # Top wall
        pygame.Rect(0, 0, OBSTACLE_THICKNESS, HEIGHT),          # Left wall
        pygame.Rect(0, HEIGHT - OBSTACLE_THICKNESS, WIDTH, OBSTACLE_THICKNESS), # Bottom wall
        pygame.Rect(WIDTH - OBSTACLE_THICKNESS, 0, OBSTACLE_THICKNESS, HEIGHT), # Right wall
    ]

    for rect in border_rects:
        body = pymunk.Body(body_type=pymunk.Body.STATIC)
        body.position = rect.center
        shape = pymunk.Poly.create_box(body, (rect.width, rect.height))
        shape.friction = 0.5
        shape.elasticity = 0.9
        shape.collision_type = WALL_COLLISION_TYPE
        current_space.add(body, shape)
        pymunk_shapes.append((body, shape))

    # Generate random inner obstacles
    inner_obstacle_rects = []
    
    # Define padding for obstacle placement to avoid overcrowding near borders or other obstacles
    obstacle_padding = ROBOT_RADIUS * 2 # Enough space for the robot to potentially pass
    
    for _ in range(num_inner_obstacles):
        attempts = 0
        while attempts < 100: # Try to place obstacle up to 100 times
            # Randomly choose if it's a horizontal or vertical wall
            is_horizontal = random.random() < 0.5
            
            if is_horizontal:
                width = random.randint(OBSTACLE_MIN_SIZE, OBSTACLE_MAX_SIZE)
                height = OBSTACLE_THICKNESS
            else: # Vertical
                height = random.randint(OBSTACLE_MIN_SIZE, OBSTACLE_MAX_SIZE)
                width = OBSTACLE_THICKNESS

            # Calculate safe placement bounds for the obstacle itself
            # Ensures obstacle doesn't go outside the main play area, considering its own size
            # And leaves some margin from the border walls
            x_min_placement = OBSTACLE_THICKNESS + obstacle_padding
            x_max_placement = WIDTH - OBSTACLE_THICKNESS - width - obstacle_padding
            y_min_placement = OBSTACLE_THICKNESS + obstacle_padding
            y_max_placement = HEIGHT - OBSTACLE_THICKNESS - height - obstacle_padding

            # Ensure valid range for random.randint, if min > max, set min to max
            x_min_placement = min(x_min_placement, x_max_placement)
            y_min_placement = min(y_min_placement, y_max_placement)

            if x_min_placement >= x_max_placement or y_min_placement >= y_max_placement:
                # If the play area is too small for any obstacle, skip this one
                attempts = 100 # Stop trying for this obstacle
                continue

            x = random.randint(x_min_placement, x_max_placement)
            y = random.randint(y_min_placement, y_max_placement)
            
            new_rect = pygame.Rect(x, y, width, height)
            
            # Check for overlap with existing obstacles (both borders and inner)
            overlap = False
            # Inflate new_rect to check for overlap with a buffer, allowing robot to pass
            check_rect = new_rect.inflate(ROBOT_RADIUS * 3, ROBOT_RADIUS * 3) # Larger buffer for checking
            
            for existing_rect in border_rects + inner_obstacle_rects:
                if check_rect.colliderect(existing_rect):
                    overlap = True
                    break
            
            if not overlap:
                inner_obstacle_rects.append(new_rect)
                break # Successfully placed
            attempts += 1
        
        # If attempts exhausted for an obstacle, it means the map is too dense.
        # We just move on without adding this specific obstacle.

    for rect in inner_obstacle_rects:
        body = pymunk.Body(body_type=pymunk.Body.STATIC)
        body.position = rect.center
        shape = pymunk.Poly.create_box(body, (rect.width, rect.height))
        shape.friction = 0.5
        shape.elasticity = 0.9
        shape.collision_type = WALL_COLLISION_TYPE
        current_space.add(body, shape)
        pymunk_shapes.append((body, shape))

    return pymunk_shapes, border_rects + inner_obstacle_rects # Return all obstacle rects for spawning check

def find_safe_spawn_position(current_space, existing_obstacle_rects):
    """
    Tries to find a safe position for the robot to spawn without overlapping obstacles.
    Returns (x, y) coordinates.
    """
    # Define the boundaries for safe spawning (away from borders)
    spawn_margin = ROBOT_RADIUS + OBSTACLE_THICKNESS + 20 # Increased margin for safety
    min_x = spawn_margin
    max_x = WIDTH - spawn_margin
    min_y = spawn_margin
    max_y = HEIGHT - spawn_margin

    if min_x >= max_x or min_y >= max_y:
        # If the spawnable area is too small or invalid due to margins
        return WIDTH // 2, HEIGHT // 2 # Fallback to center

    attempts = 0
    while attempts < 200: # Try up to 200 times to find a safe spot
        x = random.randint(min_x, max_x)
        y = random.randint(min_y, max_y)
        
        # Create a temporary circle for overlap check (robot's area)
        temp_robot_rect = pygame.Rect(x - ROBOT_RADIUS, y - ROBOT_RADIUS, 
                                      ROBOT_RADIUS * 2, ROBOT_RADIUS * 2)

        overlap = False
        # Inflate obstacle rect slightly more for collision check during spawn
        # This ensures the robot has adequate clear space upon spawning
        for obs_rect in existing_obstacle_rects:
            if temp_robot_rect.colliderect(obs_rect.inflate(ROBOT_RADIUS + 5, ROBOT_RADIUS + 5)): 
                overlap = True
                break
        
        if not overlap:
            return x, y
        attempts += 1
    
    print("Warning: Could not find a perfectly safe spawn position for the robot after many attempts. Defaulting to center.")
    return WIDTH // 2, HEIGHT // 2


def evaluate_genome(genome, config):
    """
    Evaluates the fitness of a single NEAT genome by running a robot simulation.
    The robot's behavior is controlled by the neural network represented by the genome.
    Fitness is calculated based on movement, exploration, and collision avoidance.
    """
    global space 
    
    net = neat.nn.FeedForwardNetwork.create(genome, config)
    
    # Initialize PyMunk space for each genome evaluation to ensure a fresh environment
    space = pymunk.Space()
    space.gravity = (0, 0) # No gravity for this top-down simulation
    
    # Randomize number of inner obstacles per genome evaluation
    num_inner_obstacles = random.randint(MIN_INNER_OBSTACLES, MAX_INNER_OBSTACLES)
    pymunk_obstacles_shapes, all_obstacle_rects = create_pymunk_obstacles(space, num_inner_obstacles)

    # Find a safe starting position for the robot
    start_x, start_y = find_safe_spawn_position(space, all_obstacle_rects)
    start_angle = random.uniform(-math.pi, math.pi) # Full 360 degree randomization
    
    robot = Robot(start_x, start_y, space)
    robot.body.angle = start_angle 

    # Flag to detect collision in the current step
    collision_in_this_step = [False] 
    
    # Collision handler for robot and walls
    def robot_wall_collision_begin(arbiter, current_space, data):
        """Callback for when robot begins to collide with a wall."""
        nonlocal collision_in_this_step
        collision_in_this_step[0] = True
        return True 

    handler = space.add_collision_handler(ROBOT_COLLISION_TYPE, WALL_COLLISION_TYPE)
    handler.begin = robot_wall_collision_begin

    fitness = 0
    max_steps = 500 # Maximum simulation steps for a single robot

    # For exploration reward
    visited_cells = set()
    
    for step in range(max_steps):
        dt = 1.0 / 60.0 # Time step for physics simulation (60 FPS)
        space.step(dt)

        if DRAW: # Only draw if visualization is enabled (for visualizing best genome)
            screen.fill(WHITE)
            space.debug_draw(draw_options)
            robot.draw(screen) # Draw custom robot elements on top
            
            # Display current fitness for visual debugging
            fitness_text = font.render(f"Fitness: {fitness:.2f}", True, BLACK)
            screen.blit(fitness_text, (10, 10))
            
            pygame.display.flip()
            clock.tick(60)

        inputs = robot.get_sensor_data() # Get sensor readings (distances to obstacles)
        
        output = net.activate(inputs) # Activate the neural network
        
        left_motor_speed = output[0] # Output for left motor (e.g., -1 to 1)
        right_motor_speed = output[1] # Output for right motor (e.g., -1 to 1)

        robot.update(left_motor_speed, right_motor_speed)

        # --- Fitness calculation ---
        # Sensor score: Sum of normalized sensor readings (higher is better, means less close to walls)
        sensor_score = sum(inputs) / len(inputs) 
        
        # Linear speed reward: Encourage forward movement
        current_linear_speed = robot.body.velocity.length 
        
        linear_speed_reward_component = 0.0
        if current_linear_speed >= MIN_DESIRED_SPEED:
            # Increased multiplier for linear speed reward
            linear_speed_reward_component = current_linear_speed * 1.0 # Changed from 0.7
        else:
            # Penalize more if speed is too low (below desired minimum)
            linear_speed_reward_component = (current_linear_speed - MIN_DESIRED_SPEED) * 2.0 

        # Penalize excessive turning (i.e., when motor speeds are very different)
        # Increased multiplier for angular velocity penalty
        angular_velocity_penalty = abs(left_motor_speed - right_motor_speed) * 3.0 # Changed from 2.0
        
        survival_reward = 0.1 # Small reward for each step survived

        # Exploration Reward: Reward for entering new grid cells
        current_cell_x = int(robot.body.position.x // EXPLORATION_GRID_SIZE)
        current_cell_y = int(robot.body.position.y // EXPLORATION_GRID_SIZE)
        current_cell = (current_cell_x, current_cell_y)

        exploration_reward = 0.0
        if current_cell not in visited_cells:
            visited_cells.add(current_cell)
            exploration_reward = EXPLORATION_REWARD_PER_CELL # Reward for exploring a new cell

        # Combine fitness components - TUNE THESE MULTIPLIERS!
        # Angular penalty multiplier increased from 1.5 to 2.5
        fitness += (sensor_score * 7.0) + linear_speed_reward_component - (angular_velocity_penalty * 3.0) + survival_reward + exploration_reward
        
        if collision_in_this_step[0]:
            fitness -= 500 # Very high penalty for hitting a wall to strongly discourage it
            break # End simulation for this genome if a collision occurs
        
        collision_in_this_step[0] = False # Reset collision flag for next step
        
    genome.fitness = max(0.0, fitness) # Ensures fitness is not negative. If negative scores are meaningful for sorting, remove max(0.0, ...)

    # Clean up PyMunk objects for the current genome evaluation to prevent memory leaks
    space.remove(robot.body, robot.shape)
    for body, shape in pymunk_obstacles_shapes:
        space.remove(body, shape)

class Robot:
    """Represents the robot agent in the PyMunk simulation."""
    def _init_(self, x, y, space_ref):
        self.space = space_ref 
        self.radius = ROBOT_RADIUS
        
        mass = 1
        inertia = pymunk.moment_for_circle(mass, 0, self.radius, (0, 0))
        self.body = pymunk.Body(mass, inertia)
        self.body.position = (x, y)
        self.body.friction = 0.7 # Add friction to simulate real-world movement

        self.shape = pymunk.Circle(self.body, self.radius)
        self.shape.elasticity = 0.5 # A little bounce when colliding
        self.shape.collision_type = ROBOT_COLLISION_TYPE # Assign collision type for handlers

        self.space.add(self.body, self.shape)

        # Sensor angles are calculated more precisely for the new NUM_SENSORS
        if NUM_SENSORS > 1:
            self.sensor_angles = [
                -SENSOR_ANGLE_SPREAD / 2 + i * (SENSOR_ANGLE_SPREAD / (NUM_SENSORS - 1))
                for i in range(NUM_SENSORS)
            ]
        else: # Case for 1 sensor (e.g., if NUM_SENSORS is 1, it's just straight ahead)
            self.sensor_angles = [0] 

        self.trace = [] # To draw the path the robot takes

    def get_sensor_data(self):
        """
        Performs segment queries (raycasts) to detect obstacles and return
        normalized distances for each sensor.
        """
        readings = []
        for sensor_angle in self.sensor_angles:
            angle = self.body.angle + sensor_angle # Sensor angle relative to robot's orientation
            
            # Start point of the sensor ray (just outside the robot's radius)
            p1 = self.body.position + Vec2d(math.cos(angle), math.sin(angle)) * (self.radius + 1)
            # End point of the sensor ray (at max sensor range)
            p2 = self.body.position + Vec2d(math.cos(angle), math.sin(angle)) * MAX_SENSOR_RANGE

            # Filter out the robot's own shape from collision detection
            query_filter = pymunk.ShapeFilter(mask=pymunk.ShapeFilter.ALL_MASKS() ^ (1 << (ROBOT_COLLISION_TYPE -1))) 
            
            hit = self.space.segment_query_first(p1, p2, 1, query_filter) # Query for first hit
            
            if hit:
                # Calculate distance to hit point and normalize it (0 to 1)
                distance_to_hit = hit.alpha * MAX_SENSOR_RANGE 
                readings.append(distance_to_hit / MAX_SENSOR_RANGE)
            else:
                readings.append(1.0) # No hit within MAX_SENSOR_RANGE, return full range (normalized to 1)
        return readings

    def update(self, left_speed_raw, right_speed_raw):
        """
        Updates the robot's velocity and angular velocity based on the
        normalized motor speeds from the neural network output.
        """
        # Scale the NN outputs (which are typically -1 to 1 from tanh activation in NEAT)
        # to a desired motor speed range for the robot.
        MAX_MOTOR_ACTUATION_SPEED = 75 # Max speed that a single wheel can provide
        
        left_speed = left_speed_raw * MAX_MOTOR_ACTUATION_SPEED
        right_speed = right_speed_raw * MAX_MOTOR_ACTUATION_SPEED

        # Cap speeds if they somehow exceed the max (shouldn't with tanh but good practice)
        left_speed = max(-MAX_MOTOR_ACTUATION_SPEED, min(MAX_MOTOR_ACTUATION_SPEED, left_speed))
        right_speed = max(-MAX_MOTOR_ACTUATION_SPEED, min(MAX_MOTOR_ACTUATION_SPEED, right_speed))

        # Calculate linear velocity (forward/backward movement)
        linear_vel = (left_speed + right_speed) / 2.0
        
        # Calculate angular velocity (turning) for differential drive robot
        WHEEL_SEPARATION = ROBOT_RADIUS * 2 # Assumed distance between wheels
        angular_vel = (right_speed - left_speed) / WHEEL_SEPARATION 

        # Apply calculated velocities to the robot's body
        self.body.velocity = Vec2d(math.cos(self.body.angle), math.sin(self.body.angle)) * linear_vel
        self.body.angular_velocity = angular_vel

        # Store current position for drawing trace
        self.trace.append((int(self.body.position.x), int(self.body.position.y)))
        if len(self.trace) > 100: # Keep trace length limited for performance
            self.trace.pop(0)

    def draw(self, screen_ref):
        """Draws the robot, its orientation, sensors, and movement trace on the screen."""
        p = self.body.position # Current position of the robot
        
        # Draw robot circle
        pygame.draw.circle(screen_ref, BLUE, (int(p.x), int(p.y)), int(self.radius), 2)
        
        # Draw robot orientation line (indicates "front" of the robot)
        forward_vec = Vec2d(math.cos(self.body.angle), math.sin(self.body.angle)) * self.radius
        pygame.draw.line(screen_ref, RED, (int(p.x), int(p.y)), (int(p.x + forward_vec.x), int(p.y + forward_vec.y)), 2)

        # Draw sensor lines (visualizing the raycasts)
        for sensor_angle in self.sensor_angles:
            angle = self.body.angle + sensor_angle
            p1_sensor = self.body.position + Vec2d(math.cos(angle), math.sin(angle)) * self.radius
            p2_sensor = self.body.position + Vec2d(math.cos(angle), math.sin(angle)) * MAX_SENSOR_RANGE
            
            # Re-perform the sensor query here to get the hit point for drawing
            query_filter = pymunk.ShapeFilter(mask=pymunk.ShapeFilter.ALL_MASKS() ^ (1 << (ROBOT_COLLISION_TYPE -1)))
            hit = self.space.segment_query_first(p1_sensor, p2_sensor, 1, query_filter)

            if hit:
                hit_point = hit.point
                # Draw line to the hit point and a small circle at the hit point
                pygame.draw.line(screen_ref, GREEN, (int(p1_sensor.x), int(p1_sensor.y)), (int(hit_point.x), int(hit_point.y)), 1)
                pygame.draw.circle(screen_ref, GREEN, (int(hit_point.x), int(hit_point.y)), 3) 
            else:
                # Draw full sensor line if no hit
                pygame.draw.line(screen_ref, GREEN, (int(p1_sensor.x), int(p1_sensor.y)), (int(p2_sensor.x), int(p2_sensor.y)), 1)
        
        # Draw robot trace (path history)
        if len(self.trace) > 1:
            pygame.draw.lines(screen_ref, (200, 200, 255), False, self.trace, 1)


def run(config_path):
    """
    Main function to run the NEAT training process.
    It loads the NEAT configuration, creates a population, and runs the evolution.
    """
    show_intro() # Show introduction before starting training
    config = neat.Config(
        neat.DefaultGenome,
        neat.DefaultReproduction,
        neat.DefaultSpeciesSet,
        neat.DefaultStagnation,
        config_path
    )
    pop = neat.Population(config)
    pop.add_reporter(neat.StdOutReporter(True)) # Report progress to console
    stats = neat.StatisticsReporter()
    pop.add_reporter(stats)
    
    # Checkpoint every 5 generations to save progress
    pop.add_reporter(neat.Checkpointer(5, filename_prefix='neat-checkpoint-'))

    # Run NEAT for a maximum of 50 generations
    winner = pop.run(eval_genomes, 50)
    
    # Save the best genome found to a file
    with open("best_genome.pkl", "wb") as f:
        pickle.dump(winner, f)
    print(f"Best genome saved to best_genome.pkl with fitness: {winner.fitness}")


def eval_genomes(genomes, config):
    """
    Callback function for NEAT, called for each generation to evaluate all genomes.
    """
    for genome_id, genome in genomes:
        evaluate_genome(genome, config)

def visualize_best(config_path):
    """
    Loads the best saved genome and visualizes its performance in the simulation.
    """
    global DRAW, space
    DRAW = True # Enable drawing for visualization
    
    try:
        with open("best_genome.pkl", "rb") as f:
            genome = pickle.load(f)
    except FileNotFoundError:
        print("Error: best_genome.pkl not found. Please run the training first.")
        return

    config = neat.Config(
        neat.DefaultGenome,
        neat.DefaultReproduction,
        neat.DefaultSpeciesSet,
        neat.DefaultStagnation,
        config_path
    )
    net = neat.nn.FeedForwardNetwork.create(genome, config)

    space = pymunk.Space()
    space.gravity = (0, 0)
    # For visualization, create a fixed set of obstacles (e.g., average number)
    num_vis_obstacles = (MIN_INNER_OBSTACLES + MAX_INNER_OBSTACLES) // 2 
    pymunk_obstacles_shapes, all_obstacle_rects = create_pymunk_obstacles(space, num_vis_obstacles)

    start_x, start_y = find_safe_spawn_position(space, all_obstacle_rects)
    robot = Robot(start_x, start_y, space) 
    robot.body.angle = -math.pi / 2 # Start facing up (or random if you prefer)

    collision_flag = [False] # Flag to indicate if a collision occurred for display
    def vis_collision_handler(arbiter, current_space, data):
        """Collision handler for visualization mode."""
        collision_flag[0] = True
        return True 

    handler = space.add_collision_handler(ROBOT_COLLISION_TYPE, WALL_COLLISION_TYPE)
    handler.begin = vis_collision_handler

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        dt = 1.0 / 60.0
        space.step(dt)

        screen.fill(WHITE)
        
        space.debug_draw(draw_options) # Draw PyMunk shapes
        
        inputs = robot.get_sensor_data() # Get sensor inputs for the network
        
        output = net.activate(inputs) # Get motor outputs from the network
        left_speed, right_speed = output[0], output[1]
        
        robot.update(left_speed, right_speed) # Update robot's physics
        
        robot.draw(screen) # Draw robot's custom elements

        if collision_flag[0]:
            collision_text = font.render("COLLISION!", True, RED)
            screen.blit(collision_text, (WIDTH - collision_text.get_width() - 20, 20))
            # collision_flag[0] = False # Uncomment to make it flash instead of stay on
            # If you want it to stay on until next collision, keep it commented.

        pygame.display.flip() # Update the display
        clock.tick(60) # Limit frame rate
    
    # Clean up PyMunk objects after visualization
    space.remove(robot.body, robot.shape)
    for body, shape in pymunk_obstacles_shapes:
        space.remove(body, shape)

    pygame.quit() # Uninitialize Pygame

# --- Main Execution ---
if _name_ == "_main_":
    local_dir = os.getcwd() # Get the current working directory
    config_path = os.path.join(local_dir, "neat_config.txt") # Path to NEAT configuration file

    if not os.path.exists("best_genome.pkl"):
        print("No best_genome.pkl found. Starting NEAT training...")
        # Ensure the neat_config.txt is updated with appropriate num_inputs
        # before running for the first time with increased sensors
        run(config_path)
    else:
        print("best_genome.pkl found. Skipping training, directly visualizing best genome.")
    
    visualize_best(config_path)